In [1]:
%reload_ext autoreload
%autoreload

import os
from itertools import product

import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [ ]:
os.path.exists("C:/Users/q1371/Desktop/Projects/BenchmarkDatasets")

In [2]:
norm_type = 'DecPre'
assert norm_type in ['Zscore','MinMax','DecPre']

if norm_type == 'Zscore':
    norm_num = 1
    file_type = 'ZScore'
elif norm_type == 'MinMax':
    norm_num = 2
    file_type = norm_type
elif norm_type == 'DecPre':
    norm_num = 3
    file_type = norm_type

In [3]:
root = "C:/Users/q1371/Desktop/Projects"
Trainset_path = f'{root}/BenchmarkDatasets/NoAuction/{norm_num}.NoAuction_{norm_type}/NoAuction_{norm_type}_Training'
Train_path = os.path.join(Trainset_path,f'Train_Dst_NoAuction_{file_type}_CF_1.txt')
Testset_path = f'{root}/BenchmarkDatasets/NoAuction/{norm_num}.NoAuction_{norm_type}/NoAuction_{norm_type}_Testing'
Test_path_1 = os.path.join(Testset_path,f'Test_Dst_NoAuction_{file_type}_CF_7.txt')
Test_path_2 = os.path.join(Testset_path,f'Test_Dst_NoAuction_{file_type}_CF_8.txt')
Test_path_3 = os.path.join(Testset_path,f'Test_Dst_NoAuction_{file_type}_CF_9.txt')

In [4]:
def load_raw(day: int, normalization="DecPre") -> np.array:
    index = {"DecPre": 3, "Zscore": 1}[normalization]
    root = f"C:/Users/q1371/Desktop/Projects/BenchmarkDatasets/NoAuction/{index}.NoAuction_{normalization}"
    if normalization == "Zscore":
        normalization1 = "ZScore"
    else:
        normalization1 = normalization
    if day == 1:
        return np.loadtxt(
            f"{root}/NoAuction_{normalization}_Training/Train_Dst_NoAuction_{normalization1}_CF_1.txt"
        )
    else:
        return np.loadtxt(
            f"{root}/NoAuction_{normalization}_Testing/Test_Dst_NoAuction_{normalization1}_CF_{day-1}.txt"
        )

In [5]:
# # 如何找到stock的分隔点
# # 1.每一步的值和前一步的值去取插值
# # 2. 对这些插值进行排序
# # 3.不同股票之间的插值肯定最大
# # 4.找到差值最大的五个分隔点
# diff = np.diff(a_raw[0],prepend=np.inf)

# diff = np.abs(diff)
# sort_idx = np.argsort(diff)
# split_point = np.sort(sort_idx[-5:])
# split_point = np.append(split_point,[a_raw.shape[1]])
# # for i in range(len(sort_idx)):
# #     print(sort_idx[i])
# split_point

In [6]:
def split(array, n_stocks=5):
    boundaries = np.sort(
        np.argsort(np.abs(np.diff(array[0], prepend=np.inf)))[-n_stocks :]
    )
    boundaries = np.append(boundaries,[array.shape[1]])
    return tuple(array[:, boundaries[i] : boundaries[i + 1]] for i in range(n_stocks))

In [7]:
def to_dataframe(array) -> pd.DataFrame:
    data = {}

    for level in range(10):
        data[f"PRICE_ASK_{level}"] = array[4 * level]
    for level in range(10):
        data[f"PRICE_BID_{level}"] = array[4 * level + 2]
    for level in range(10):
        data[f"VOLUME_ASK_{level}"] = array[4 * level + 1]
    for level in range(10):
        data[f"VOLUME_BID_{level}"] = array[4 * level + 3]
    data[f"LABEL_1TICK"] = array[-5] - 1
    data[f"LABEL_2TICK"] = array[-4] - 1
    data[f"LABEL_3TICK"] = array[-3] - 1 
    data[f"LABEL_5TICK"] = array[-2] - 1
    data[f"LABEL_10TICK"] = array[-1]- 1

    return pd.DataFrame(data)

# 3 --> 2
# 2 --> 1
# 1 --> 0

In [8]:
def revert_decimal_normalization(df, max_denom=10 ** 8):
    assert (df.iloc[:, :-5].values != 0).all()

    df.iloc[:, :-5] = df.iloc[:, :-5] * max_denom
    df.iloc[:, :-5] = df.iloc[:, :-5].round()
    df.iloc[:, :-5] = df.iloc[:, :-5].astype(int)
    for c in df.columns[:-5]:
        while (df.loc[:, c] % 10 == 0).all():
            df.loc[:, c] //= 10
    return df

In [9]:
dfs = {i_stock: {} for i_stock in range(1, 5 + 1)}

for day in tqdm(range(1, 10 + 1)):
    a_raw = load_raw(day)
    for i_stock, a in enumerate(split(a_raw)):
        dfs[i_stock + 1][day] = to_dataframe(a)

100%|██████████| 10/10 [00:56<00:00,  5.65s/it]


In [10]:
listdf = []

for i_stock, day in tqdm(list(product(range(1, 5 + 1), range(1, 10 + 1)))):
    df = dfs[i_stock][day]
    df = revert_decimal_normalization(df)
    df["STOCK"] = i_stock
    df["DAY"] = day
    df = df[list(df.columns[-2:]) + list(df.columns[:-2])]

    listdf.append(df)

dataframe = pd.concat(listdf, axis=0)
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.astype(int)
dataframe.to_csv(f"data_denorm.csv")

100%|██████████| 50/50 [00:19<00:00,  2.57it/s]


In [11]:
dataframe

,STOCK,DAY,PRICE_ASK_0,PRICE_ASK_1,PRICE_ASK_2,PRICE_ASK_3,PRICE_ASK_4,PRICE_ASK_5,PRICE_ASK_6,PRICE_ASK_7,...,VOLUME_BID_5,VOLUME_BID_6,VOLUME_BID_7,VOLUME_BID_8,VOLUME_BID_9,LABEL_1TICK,LABEL_2TICK,LABEL_3TICK,LABEL_5TICK,LABEL_10TICK
0,1,1,2615,2618,2619,2620,2621,2623,2625,2626,...,100,143,134,123,128,1,1,1,1,1
1,1,1,2615,2619,2620,2621,2625,2626,2629,2633,...,159,100,143,134,123,1,1,1,1,1
2,1,1,2614,2615,2617,2619,2620,2621,2625,2626,...,159,100,143,134,123,2,2,1,1,1
3,1,1,2614,2617,2619,2620,2621,2625,2629,2633,...,100,143,134,123,128,1,1,2,1,1
4,1,1,2614,2617,2619,2620,2621,2625,2629,2633,...,100,143,134,123,128,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394332,5,10,3783,3784,3785,3786,3787,3788,3789,3790,...,398,511,180,1446,1209,1,1,1,1,1
394333,5,10,3783,3784,3785,3786,3787,3788,3789,3790,...,398,511,180,1446,1209,1,1,1,1,1
394334,5,10,3783,3784,3785,3786,3787,3788,3789,3790,...,300,398,511,180,1446,1,1,1,1,1
394335,5,10,3783,3784,3785,3786,3787,3788,3789,3790,...,380,398,511,180,1446,1,1,1,1,1


In [12]:
dfs = {i_stock: {} for i_stock in range(1, 5 + 1)}

for day in tqdm(range(1, 10 + 1)):
    a_raw = load_raw(day, normalization="Zscore")
    for i_stock, a in enumerate(split(a_raw)):
        dfs[i_stock + 1][day] = to_dataframe(a)

100%|██████████| 10/10 [00:55<00:00,  5.56s/it]


In [13]:
listdf = []

for i_stock, day in tqdm(list(product(range(1, 5 + 1), range(1, 10 + 1)))):
    df = dfs[i_stock][day]
    # df = revert_decimal_normalization(df)
    df["STOCK"] = i_stock
    df["DAY"] = day
    df = df[list(df.columns[-2:]) + list(df.columns[:-2])]

    listdf.append(df)

dataframe = pd.concat(listdf, axis=0)
dataframe = dataframe.reset_index(drop=True)
# dataframe = dataframe.astype(int)
dataframe.iloc[:, -5:] = dataframe.iloc[:, -5:].astype(int)
dataframe.to_csv(f"data_no_date.csv")
# dataframe.iloc[:100000].to_csv(f"data_zscore1.csv")
# dataframe.iloc[100000:].to_csv(f"data_zscore2.csv")

100%|██████████| 50/50 [00:00<00:00, 253.81it/s]


In [14]:
dataframe

,STOCK,DAY,PRICE_ASK_0,PRICE_ASK_1,PRICE_ASK_2,PRICE_ASK_3,PRICE_ASK_4,PRICE_ASK_5,PRICE_ASK_6,PRICE_ASK_7,...,VOLUME_BID_5,VOLUME_BID_6,VOLUME_BID_7,VOLUME_BID_8,VOLUME_BID_9,LABEL_1TICK,LABEL_2TICK,LABEL_3TICK,LABEL_5TICK,LABEL_10TICK
0,1,1,0.408275,0.409877,0.409493,0.409170,0.408796,0.409242,0.409436,0.408336,...,-0.784121,-0.718656,-0.573157,-0.521503,-0.464096,1,1,1,1,1
1,1,1,0.408275,0.410898,0.410513,0.410191,0.412876,0.412301,0.413513,0.415468,...,-0.758706,-0.737892,-0.569944,-0.518372,-0.465155,1,1,1,1,1
2,1,1,0.407254,0.406815,0.407452,0.408150,0.407776,0.407202,0.409436,0.408336,...,-0.758706,-0.737892,-0.569944,-0.518372,-0.465155,2,2,1,1,1
3,1,1,0.407254,0.408857,0.409493,0.409170,0.408796,0.411282,0.413513,0.415468,...,-0.784121,-0.718656,-0.573157,-0.521503,-0.464096,1,1,2,1,1
4,1,1,0.407254,0.408857,0.409493,0.409170,0.408796,0.411282,0.413513,0.415468,...,-0.784121,-0.718656,-0.573157,-0.521503,-0.464096,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394332,5,10,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,...,-0.679060,-0.577830,-0.607661,-0.148597,-0.239478,1,1,1,1,1
394333,5,10,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,...,-0.679060,-0.577830,-0.607661,-0.148597,-0.239478,1,1,1,1,1
394334,5,10,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,...,-0.721177,-0.628241,-0.483058,-0.516647,-0.186564,1,1,1,1,1
394335,5,10,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,...,-0.686796,-0.628241,-0.483058,-0.516647,-0.186564,1,1,1,1,1


In [ ]:
dataframe.loc[dataframe['STOCK']==1]

In [ ]:
day_begin = ["2010-06-01 07:30:00.000",
             "2010-06-02 07:30:00.000",
             "2010-06-03 07:30:00.000",
             "2010-06-04 07:30:00.000",
             "2010-06-07 07:30:00.000",
             "2010-06-08 07:30:00.000",
             "2010-06-09 07:30:00.000",
             "2010-06-10 07:30:00.000",
             "2010-06-11 07:30:00.000",
             "2010-06-14 07:30:00.000"]
day_end =["2010-06-01 15:25:00.000",
              "2010-06-02 15:25:00.000",
              "2010-06-03 15:25:00.000",
              "2010-06-04 15:25:00.000",
              "2010-06-07 15:25:00.000",
              "2010-06-08 15:25:00.000",
              "2010-06-09 15:25:00.000",
              "2010-06-10 15:25:00.000",
              "2010-06-11 15:25:00.000",
              "2010-06-14 15:25:00.000"]


# day_one_begin = datetime.strptime(day_begin[0], "%Y-%m-%d %H:%M:%S.%f")
# day_one_end = datetime.strptime(day_end[0], "%Y-%m-%d %H:%M:%S.%f")

In [ ]:
len(day_end)

In [ ]:
# step = (day_one_end - day_one_begin)/3454

In [ ]:
# new = day_one_begin + step

In [ ]:
# print(new)

In [ ]:
all_date = []
for j in tqdm(range(1,11)):
    for k in range(1,6):
        for i in range(len(dataframe.loc[(dataframe['STOCK']==k)&(dataframe['DAY']==j)])):
            d_begin = datetime.strptime(day_begin[j-1], "%Y-%m-%d %H:%M:%S.%f")
            d_end = datetime.strptime(day_end[j-1], "%Y-%m-%d %H:%M:%S.%f")
            step = (d_end - d_begin)/len(dataframe.loc[(dataframe['STOCK']==k)&(dataframe['DAY']==j)])
            all_date.append(d_begin + i*step)


In [ ]:
# all_date = np.array(all_date)
# np.save('date',all_date)

In [17]:
all_date = np.load("date.npy",allow_pickle=True)

In [18]:
date_df = pd.DataFrame(all_date)

In [19]:
date_df

,0
0,2010-06-01 07:30:00.000000
1,2010-06-01 07:30:08.251303
2,2010-06-01 07:30:16.502606
3,2010-06-01 07:30:24.753909
4,2010-06-01 07:30:33.005212
...,...
394332,2010-06-14 15:24:49.344399
394333,2010-06-14 15:24:51.474762
394334,2010-06-14 15:24:53.605125
394335,2010-06-14 15:24:55.735488


In [20]:
date_df.columns = ['date']

In [21]:
date_df

,date
0,2010-06-01 07:30:00.000000
1,2010-06-01 07:30:08.251303
2,2010-06-01 07:30:16.502606
3,2010-06-01 07:30:24.753909
4,2010-06-01 07:30:33.005212
...,...
394332,2010-06-14 15:24:49.344399
394333,2010-06-14 15:24:51.474762
394334,2010-06-14 15:24:53.605125
394335,2010-06-14 15:24:55.735488


In [22]:
dataframe.insert(1, "date", date_df)

In [23]:
all_cols = dataframe.columns.values.tolist()
del_col = ['STOCK','DAY']

In [24]:
for i in range(len(del_col)):   
    all_cols.remove(del_col[i])

In [25]:
new_data = pd.DataFrame(dataframe,columns=all_cols)

In [26]:
mid_price = (new_data['PRICE_ASK_0'] + new_data['PRICE_BID_0'])/2

In [27]:
new_data.insert(46, "MID_PRICE", mid_price)

In [28]:
new_data

,date,PRICE_ASK_0,PRICE_ASK_1,PRICE_ASK_2,PRICE_ASK_3,PRICE_ASK_4,PRICE_ASK_5,PRICE_ASK_6,PRICE_ASK_7,PRICE_ASK_8,...,VOLUME_BID_6,VOLUME_BID_7,VOLUME_BID_8,VOLUME_BID_9,LABEL_1TICK,LABEL_2TICK,LABEL_3TICK,LABEL_5TICK,LABEL_10TICK,MID_PRICE
0,2010-06-01 07:30:00.000000,0.408275,0.409877,0.409493,0.409170,0.408796,0.409242,0.409436,0.408336,0.408945,...,-0.718656,-0.573157,-0.521503,-0.464096,1,1,1,1,1,0.405926
1,2010-06-01 07:30:08.251303,0.408275,0.410898,0.410513,0.410191,0.412876,0.412301,0.413513,0.415468,0.417091,...,-0.737892,-0.569944,-0.518372,-0.465155,1,1,1,1,1,0.405926
2,2010-06-01 07:30:16.502606,0.407254,0.406815,0.407452,0.408150,0.407776,0.407202,0.409436,0.408336,0.408945,...,-0.737892,-0.569944,-0.518372,-0.465155,2,2,1,1,1,0.405415
3,2010-06-01 07:30:24.753909,0.407254,0.408857,0.409493,0.409170,0.408796,0.411282,0.413513,0.415468,0.417091,...,-0.718656,-0.573157,-0.521503,-0.464096,1,1,2,1,1,0.405415
4,2010-06-01 07:30:33.005212,0.407254,0.408857,0.409493,0.409170,0.408796,0.411282,0.413513,0.415468,0.417091,...,-0.718656,-0.573157,-0.521503,-0.464096,0,0,0,1,1,0.405415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394332,2010-06-14 15:24:49.344399,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,1.456363,...,-0.577830,-0.607661,-0.148597,-0.239478,1,1,1,1,1,1.465332
394333,2010-06-14 15:24:51.474762,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,1.456363,...,-0.577830,-0.607661,-0.148597,-0.239478,1,1,1,1,1,1.465332
394334,2010-06-14 15:24:53.605125,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,1.456363,...,-0.628241,-0.483058,-0.516647,-0.186564,1,1,1,1,1,1.465332
394335,2010-06-14 15:24:55.735488,1.462738,1.462118,1.461516,1.461029,1.460577,1.459931,1.459059,1.457881,1.456363,...,-0.628241,-0.483058,-0.516647,-0.186564,1,1,1,1,1,1.465332


In [29]:
new_data.to_csv(f"data.csv")

In [179]:
url = "https://raw.githubusercontent.com/simaki/fi2010/main/data/data.csv"
df = pd.read_csv(url, index_col=0)

In [178]:
df

""
version https://git-lfs.github.com/spec/v1
oid sha256:26d04dcfbfe2ff7400cee47588d5758c7ab99df50339da4cca12919255ffc867
size 194790126
